# Instructions

Ce notebook ne doit contenir que votre script servant à utiliser vos modèles **entrainés** en les testant sur le dataset *fake-test.csv*. Nous devons pouvoir l'exécuter en cliquant sur *Exécution -> Tout exécuter*.

Nous utiliserons ce script pour évaluer votre modèle sur nos propres données d'évaluation par la suite.


In [3]:
from transformers import AutoModelForSequenceClassification,Trainer, AutoTokenizer
import torch
from datasets import load_dataset
import numpy as np
import evaluate

# Load the model and tokenizer
model = AutoModelForSequenceClassification.from_pretrained("saved_model").cuda()
tokenizer = AutoTokenizer.from_pretrained("saved_model")
ds_evaluate = load_dataset("csv", data_files="fake_test.csv", split='train')

In [4]:
def preprocess_function(examples):
    return tokenizer(examples['data'], truncation=True, padding=True, max_length=512)

accuracy = evaluate.load("accuracy")
tokenized_evaluate = ds_evaluate.map(preprocess_function, batched=True)


trainer = Trainer(model= model)
predictions = trainer.predict(tokenized_evaluate)
predicted_labels = np.argmax(predictions.predictions, axis=1)

true_labels = ds_evaluate['label']
results = accuracy.compute(predictions=predicted_labels, references=true_labels)
print("Evaluation Results:", results)

100%|██████████| 61/61 [00:07<00:00,  7.90it/s]

Evaluation Results: {'accuracy': 0.9588477366255144}


In [ ]:
# Tester le modèle avec du text 

# Vérifier si CUDA est disponible
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print("Using device:", device)

# Exemple de texte non labellisé
unlabeled_data = [
    "« Les participants au concours de robotique ont présenté plus de 50 projets innovants lors de la finale régionale » a annoncé le comité d'organisation dans son rapport annuel."
]

# Étape 1 : Prétraitement
def preprocess_function(examples):
    return tokenizer(examples, truncation=True, padding=True, max_length=512, return_tensors="pt")

tokenized_data = preprocess_function(unlabeled_data)

# Déplacer les données sur l'appareil disponible
tokenized_data = {key: val.to(device) for key, val in tokenized_data.items()}

# Étape 2 : Charger le modèle et le déplacer sur le même appareil
model.to(device)
model.eval()  # Mode évaluation

# Étape 3 : Prédictions
with torch.no_grad():
    outputs = model(**tokenized_data)  # Entrer directement les données tokenisées
    predictions = np.argmax(outputs.logits.cpu().numpy(), axis=1)  # Prédictions

# Étape 4 : Résultat des prédictions
print("Predicted Labels:", predictions)

# Décoder les labels (si id2label est défini)
id2label = model.config.id2label
decoded_labels = [id2label[label] for label in predictions]
print("Decoded Labels:", decoded_labels)
